<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/Poly-Encoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Jul  5 06:16:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers datasets folium==0.2.1

In [ ]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 108 (delta 28), reused 20 (delta 8), pack-reused 57
Receiving objects: 100% (108/108), 36.04 KiB | 9.01 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [ ]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG

In [ ]:
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture bi
!python3 run.py --bert_model 'klue/bert-base' --output_dir 'chatbot_output/' --train_dir '감성대화챗봇데이터/' --train_file 'train_data_source.pickle' --dev_file 'val_data_source.pickle' --test_file 'test_data_source.pickle' --architecture poly --poly_m 16
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture cross

Namespace(adam_epsilon=1e-08, architecture='poly', bert_model='klue/bert-base', dev_file='val_data_source.pickle', eval=False, eval_batch_size=32, fp16=False, fp16_opt_level='O1', gpu=0, gradient_accumulation_steps=1, learning_rate=5e-05, max_contexts_length=128, max_grad_norm=1.0, max_response_length=32, model_type='bert', num_train_epochs=10.0, output_dir='chatbot_output/', poly_m=16, print_freq=100, seed=12345, test_file='test_data_source.pickle', train_batch_size=32, train_dir='감성대화챗봇데이터/', train_file='train_data_source.pickle', use_pretrain=False, warmup_steps=100, weight_decay=0.01)
Train dir: 감성대화챗봇데이터/
Output dir: chatbot_output/
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_r

In [ ]:
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture bi --eval
!python3 run.py --bert_model 'klue/bert-base' --output_dir 'chatbot_output/' --train_dir '감성대화챗봇데이터/' --train_file 'train_data_source.pickle' --dev_file 'val_data_source.pickle' --test_file 'test_data_source.pickle' --architecture poly --poly_m 16 --eval
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture poly --poly_m 16 --eval
# !python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture cross --eval

Namespace(adam_epsilon=1e-08, architecture='bi', bert_model='bert_model/', eval=True, eval_batch_size=32, fp16=False, fp16_opt_level='O1', gpu=0, gradient_accumulation_steps=1, learning_rate=5e-05, max_contexts_length=128, max_grad_norm=1.0, max_response_length=32, model_type='bert', num_train_epochs=10.0, output_dir='output_dstc7/', poly_m=0, print_freq=100, seed=12345, train_batch_size=32, train_dir='dstc7/', use_pretrain=True, warmup_steps=100, weight_decay=0.01)
Train dir: dstc7/
Output dir: output_dstc7/
Loading parameters from output_dstc7/bi_0_pytorch_model.bin
{'eval_loss': 2.34547645971179, 'R1': 0.481, 'R2': 0.581, 'R5': 0.685, 'R10': 0.77, 'MRR': 0.581299067078589}


In [ ]:
# !cp -r /content/Poly-Encoder/output_dstc7 '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/Poly Encoder'

In [ ]:
PATH = '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/Poly Encoder/output_dstc7'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder

PATH = '/content/Poly-Encoder/chatbot_output/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

In [ ]:
# sentences = ['This framework generates embeddings for each input sentence']

In [ ]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [ ]:
with torch.no_grad():
    model.eval()
    model_output = model.bert(**encoded_input)

In [ ]:
model_output[0].shape

torch.Size([3, 25, 768])

In [ ]:
model_output[0][:,0,:]

tensor([[ 0.0931, -0.1231, -0.0292,  ..., -0.0014, -0.1432,  0.1350],
        [ 0.0868, -0.1291, -0.0281,  ..., -0.0047, -0.1368,  0.1317],
        [ 0.0876, -0.1231, -0.0293,  ..., -0.0074, -0.1360,  0.1305]])

In [ ]:
model_output = model.bert(**encoded_input)

In [ ]:
model_output[0][:,0,:]

tensor([[ 0.0931, -0.1231, -0.0292,  ..., -0.0014, -0.1432,  0.1350],
        [ 0.0868, -0.1291, -0.0281,  ..., -0.0047, -0.1368,  0.1317],
        [ 0.0876, -0.1231, -0.0293,  ..., -0.0074, -0.1360,  0.1305]],
       grad_fn=<SliceBackward0>)

In [ ]:
model.dot_attention()

In [ ]:
from inspect import signature
model.bert